In [6]:
import spacy
import re

In [4]:
corpus = open('corpus.txt',encoding='utf-8').readlines()

In [8]:
nlp = spacy.load('de_core_news_md', disable=['ner', 'parser'])

In [9]:
def cleaning(doc):
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

In [21]:
brief_cleaning = [re.sub("[^A-ZAAÖUa-zäöü#+']+", ' ', str(row)).lower() for row in corpus]

In [24]:
from time import time
t = time()
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.43 mins


In [28]:
from gensim.models.phrases import Phrases, Phraser

In [32]:
sent = [row.split()  for row in txt if row]

In [34]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

In [35]:
bigram = Phraser(phrases)

In [36]:
sentences = bigram[sent]

In [62]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [63]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]

In [130]:
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [149]:
model.save('model.model')